# Prediction

## importation des données

In [2]:
import pandas as pd
import numpy as np

In [3]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pd.read_csv("price_dataset1.csv",index_col =0)
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,969,73,56,amsterdam,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
1,760,154,196,amsterdam,44,dutch,0,111612,Yin Yang,Royal Lotus,0,0,0
2,570,93,112,amsterdam,44,dutch,0,111612,Independant,Independant,1,0,0
3,835,97,113,amsterdam,44,dutch,0,111612,Independant,Independant,0,0,0
4,680,131,55,amsterdam,44,dutch,0,111612,Independant,Independant,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,177,164,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,0,1,0
89,47,163,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,1,0,0
90,187,168,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,1,1,0
91,136,140,0,vilnius,0,greek,0,111612,Boss Western,Boss Western,0,0,0


In [4]:
# def split_dataset(dataset, test_ratio=0.30):
#   """Splits a panda dataframe in two."""
#   test_indices = np.random.rand(len(dataset)) < test_ratio
#   return dataset[~test_indices], dataset[test_indices]


In [5]:
# train_ds_pd, test_ds_pd = split_dataset(pricing_requests)

In [6]:
import tensorflow_decision_forests as tfdf

2022-11-23 10:43:51.023438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 10:43:51.122244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/insa/lib:$LD_LIBRARY_PATH
2022-11-23 10:43:51.122264: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 10:43:53.076625: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot o

In [7]:
label ="price"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pricing_requests, label=label, task=tfdf.keras.Task.REGRESSION)
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

2022-11-23 10:43:57.295966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/insa/lib:$LD_LIBRARY_PATH
2022-11-23 10:43:57.295998: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 10:43:57.296017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (insa-12588): /proc/driver/nvidia/version does not exist
2022-11-23 10:43:57.296285: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Tuner potentiel

In [8]:
# # Configure the tuner.

# # Create a Random Search tuner with 50 trials.
# tuner = tfdf.tuner.RandomSearch(num_trials=50)

# # Define the search space.
# #
# # Adding more parameters generaly improve the quality of the model, but make
# # the tuning last longer.

# tuner.choice("min_examples", [2, 5, 7, 10])
# tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# # Some hyper-parameters are only valid for specific values of other
# # hyper-parameters. For example, the "max_depth" parameter is mostly useful when
# # "growing_strategy=LOCAL" while "max_num_nodes" is better suited when
# # "growing_strategy=BEST_FIRST_GLOBAL".

# local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
# local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# # merge=True indicates that the parameter (here "growing_strategy") is already
# # defined, and that new values are added to it.
# global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
# global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# tuner.choice("use_hessian_gain", [True, False])
# tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
# tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

# # Uncomment some (or all) of the following hyper-parameters to increase the
# # quality of the search. The number of trial should be increased accordingly.

# # tuner.choice("split_axis", ["AXIS_ALIGNED"])
# # oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
# # oblique_space.choice("sparse_oblique_normalization",
# #                      ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
# # oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
# # oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])


## Compilation et prédiction

In [9]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.compile(metrics=["mse"])
model.fit(x=train_ds, verbose =0)


Use /tmp/tmpglckpswa as temporary training directory
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
[INFO 2022-11-23T10:44:09.20857423+01:00 kernel.cc:1175] Loading model from path /tmp/tmpglckpswa/model/ with prefix 910c681a45c341e6
[INFO 2022-11-23T10:44:13.25195278+01:00 abstract_model.cc:1306] Engine "RandomForestOptPred" built
[INFO 2022-11-23T10:44:13.251982737+01:00 kernel.cc:1021] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
# evaluation = model.evaluate(test_ds ,return_dict=True)
# si on a split nos données

# print(evaluation)

In [11]:
test = pd.read_csv("test_set.csv",index_col=0)

test = test.join(hotels, on=['hotel_id', 'city'])

test = test.drop(["order_requests"],axis =1)

test_df = tfdf.keras.pd_dataframe_to_tf_dataset(test)

A =model.predict(test_df)

7/7 [==============================] - 1s 82ms/step


In [12]:
Submission = pd.DataFrame(A)

Submission.to_csv("submission1.csv",header = ["price"],index_label="index")